<a href="https://colab.research.google.com/github/ipietri/w266_Final_Project/blob/master/notebooks/RtGender-Notebooks/RtGender_Annotations_Sentiment_Easy_Data_Augmentation_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RtGender - Annotations - Sentiment w/ Easy Data Augmentation Techniques
[Source](https://github.com/jasonwei20/eda_nlp)

[Paper that explores these techniques](https://arxiv.org/abs/1901.11196)


In [8]:
try:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  path = r'/content/drive/MyDrive/w266'
except ModuleNotFoundError:
  path = r'data'

Mounted at /content/drive


<a id='section01'></a>
## Load Modules

In [9]:
!pip install -U nltk
import nltk; nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
%%capture
#!pip install transformers==3.0.2
!pip install -q transformers

In [11]:
%%capture
!pip install datasets
import datasets 
from datasets import load_dataset, Dataset, DatasetDict

In [12]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn import metrics
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
#from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import logging
logging.basicConfig(level=logging.ERROR)

In [13]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

<a id='section02'></a>
## Import and Reshape Data

In [75]:
train_df = pd.read_csv('/content/drive/MyDrive/w266/annotations_train.csv')
dev_df = pd.read_csv('/content/drive/MyDrive/w266/annotations_dev.csv')

print('train_shape: ',train_df.shape)
print('dev_shape: ',dev_df.shape)

train_shape:  (10746, 8)
dev_shape:  (2303, 8)


In [76]:
# there are NaNs in the dev dataset remove 
nan_values = dev_df[dev_df.isna().any(axis=1)] 
print(nan_values)

# return without missing values in response_text
dev_df.dropna(subset = ["response_text"], inplace=True)

print("Train shape", train_df.shape)
print("Dev shape", dev_df.shape)

      Unnamed: 0         source op_gender  ... sentiment   relevance label
830         2576  facebook_wiki         M  ...   Neutral  Irrelevant     1
1664        2722  facebook_wiki         W  ...   Neutral  Irrelevant     1

[2 rows x 8 columns]
Train shape (10746, 8)
Dev shape (2301, 8)


In [77]:
print("Unique sentiments: ", train_df['sentiment'].unique())

Unique sentiments:  ['Positive' 'Mixed' 'Neutral' 'Negative']


In [78]:
# Change Mixed to Neutral
train_df['sentiment'].replace({"Mixed":"Neutral"}, inplace = True)
dev_df['sentiment'].replace({"Mixed":"Neutral"}, inplace = True)

# Run through Easy Data Augmentation Techniques package
[augment](https://github.com/jasonwei20/eda_nlp/blob/master/code/augment.py)


#### Import Modules

In [79]:
# import eda script from github
!git clone https://github.com/jasonwei20/eda_nlp

fatal: destination path 'eda_nlp' already exists and is not an empty directory.


In [80]:
import sys
# sys.path is a list of absolute path strings
sys.path.append('/content/eda_nlp/code/')

from eda import *


#### Random deletion
Randomly delete words from the sentence with probability p


In [115]:
# apply the main data augmentation function in the eda module

augmented_sentences = []

for sentence in train_df['response_text']:
  words = sentence.split(' ')
  a_words = random_deletion(words, 0.1)
  augmented_sentences.append(' '.join(a_words))


#### Replace with Synonyms

In [141]:
augmented_sentences_2 = []
alpha_sr=0.1

if (alpha_sr > 0):
  for sentence in augmented_sentences:
    words = sentence.split(' ')
    num_words = len(sentence)
    n_sr = max(1, int(alpha_sr*num_words))
    a_words = synonym_replacement(words, n_sr)
    augmented_sentences_2.append(' '.join(a_words))

In [143]:
# add to train df
train_df_aug = train_df
train_df_aug['augmented_response_text'] = augmented_sentences_2
train_df_aug

,Unnamed: 0,source,op_gender,post_text,response_text,sentiment,relevance,label,augmented_response_text
0,3845,facebook_congress,W,Im reading the 3/1 GAO report that finds billi...,Thank you Congresswoman Bass. Keep up the grea...,Positive,ContentPoster,2,thank you congresswoman Bass. donjon up the sm...
1,9743,fitocracy,M,Being followed by the famous DBJ? Quite an honor.,"Well, I am very honored you feel so honored",Positive,Content,2,"Well, one am very observe you sense observe"
2,13041,ted,W,"Penelope Boston gave a talk about Planets, exp...",Her opinions seem driven by wishful thinking. ...,Neutral,Content,1,Her belief look impelled by aspiring thinking....
3,4265,facebook_congress,W,Congress must act to help the 41 million Ameri...,There's no other way out of the enormity excep...,Positive,Content,2,There's other way of life out of the outrageou...
4,13145,ted,W,"Pardis Sabeti gave a talk about Africa, big pr...",What were the benefits of the larger community...,Neutral,Content,1,What were the welfare of the magnanimous She d...
...,...,...,...,...,...,...,...,...,...
10741,14128,ted,M,"Bjarke Ingels gave a talk about architecture, ...",Brillant!! Ingels has a terrific future ahead ...,Positive,Content,2,Brillant!! Ingels has a forrader of The reflec...
10742,5589,facebook_congress,W,I was honored to meet with Eliseo Medina and F...,The Democrats view this as another way to use ...,Negative,Content,0,The democrat some other agency to habit taxpay...
10743,10672,reddit,W,SO YOU LIKE STACKING CUPS?! DO WE HAVE A GREAT...,Is this real?? Well at least this kid will be ...,Neutral,Content,1,constitute this real?? wellspring at to the lo...
10744,4839,facebook_congress,M,Try this Brian Schatz FB bumper sticker - an e...,EH BRIAN WEA MY STICKA N WAT OBAMA STAY ON UM ...,Neutral,Irrelevant,1,EH BRIAN WEA MY STICKA normality WAT OBAMA rem...


# Transform to HuggingFace friendly format

In [144]:
# change to dataset to work with Huggingface transformer & remove unused columns
columns_to_remove = ['op_gender', 'source', 'Unnamed: 0', 'relevance', 'sentiment','post_text']

from datasets import load_dataset
train_dataset = Dataset.from_pandas(train_df_aug)
dev_dataset = Dataset.from_pandas(dev_df)

train_dataset = train_dataset.remove_columns(column_names= columns_to_remove)
dev_dataset = dev_dataset.remove_columns(column_names= columns_to_remove)
dev_dataset = dev_dataset.remove_columns(column_names= '__index_level_0__')


# rename sentiment to labels
# train_dataset = train_dataset.rename_column("sentiment", "label")
# dev_dataset = dev_dataset.rename_column("sentiment", "label")

In [145]:
# combine into a DataDictionary for huggingface use
rtg_dataset = DatasetDict({
    'train': train_dataset,
    'dev': dev_dataset 
})

rtg_dataset

DatasetDict({
    train: Dataset({
        features: ['response_text', 'label', 'augmented_response_text'],
        num_rows: 10746
    })
    dev: Dataset({
        features: ['response_text', 'label'],
        num_rows: 2301
    })
})

## Tokenize

In [146]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [147]:
from transformers import AutoTokenizer

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["response_text"], padding=True, truncation=True)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/

In [148]:
rtg_encoded = rtg_dataset.map(tokenize, batched=True, batch_size=None)
rtg_encoded['train'].features

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

{'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'augmented_response_text': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'label': Value(dtype='int64', id=None),
 'response_text': Value(dtype='string', id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

# Model

In [149]:
from transformers import AutoModelForSequenceClassification
num_labels = 3
model = (AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device))

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size":

In [150]:
rtg_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])
rtg_encoded["dev"].features

{'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'label': Value(dtype='int64', id=None),
 'response_text': Value(dtype='string', id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [151]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [152]:
from transformers import Trainer, TrainingArguments

batch_size = 8
logging_steps = len(rtg_encoded["train"]) // batch_size
training_args = TrainingArguments(output_dir="results",
                                  num_train_epochs=2,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="f1",
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  save_strategy="epoch",
                                  disable_tqdm=False
                                  )

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [153]:
accuracy_list = []
weighted_f1_score_list = []

for i in range(2):
  try:
    del trainer
    del results
  except: pass


  trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=rtg_encoded["train"],
                  eval_dataset=rtg_encoded["dev"])
  trainer.train()
  results = trainer.evaluate()
  # append to lists
  accuracy_list.append(results.get('eval_accuracy'))
  weighted_f1_score_list.append(results.get("eval_f1"))
  print(f'---------------------------Iteration {i} Complete---------------------------\n')

print("accuracy: ", accuracy_list)
print("weighted f1 score: ", weighted_f1_score_list)

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: response_text, augmented_response_text.
***** Running training *****
  Num examples = 10746
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2688


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.669500,0.639596,0.730987,0.732118
2,0.500500,0.702909,0.739678,0.735969


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: response_text.
***** Running Evaluation *****
  Num examples = 2301
  Batch size = 8
Saving model checkpoint to results/checkpoint-1344
Configuration saved in results/checkpoint-1344/config.json
Model weights saved in results/checkpoint-1344/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: response_text.
***** Running Evaluation *****
  Num examples = 2301
  Batch size = 8
Saving model checkpoint to results/checkpoint-2688
Configuration saved in results/checkpoint-2688/config.json
Model weights saved in results/checkpoint-2688/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from results/checkpoint-2688 (score: 0.7359692606192756).
The following columns i

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: response_text, augmented_response_text.
***** Running training *****
  Num examples = 10746
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2688


---------------------------Iteration 0 Complete---------------------------



Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.339800,1.127476,0.720122,0.722086
2,0.288500,1.350583,0.719687,0.715585


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: response_text.
***** Running Evaluation *****
  Num examples = 2301
  Batch size = 8
Saving model checkpoint to results/checkpoint-1344
Configuration saved in results/checkpoint-1344/config.json
Model weights saved in results/checkpoint-1344/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: response_text.
***** Running Evaluation *****
  Num examples = 2301
  Batch size = 8
Saving model checkpoint to results/checkpoint-2688
Configuration saved in results/checkpoint-2688/config.json
Model weights saved in results/checkpoint-2688/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from results/checkpoint-1344 (score: 0.7220863025692668).
The following columns i

---------------------------Iteration 1 Complete---------------------------

accuracy:  [0.7396784006953498, 0.7201216862233811]
weighted f1 score:  [0.7359692606192756, 0.7220863025692668]


In [154]:
import statistics

print("Mean Accuracy: ",round(statistics.mean(accuracy_list),3))
print("StDev Accuracy: ",round(statistics.stdev(accuracy_list),3))

print("Mean F1: ",round(statistics.mean(weighted_f1_score_list),3))
print("StDev F1: ",round(statistics.stdev(weighted_f1_score_list),3))

Mean Accuracy:  0.73
StDev Accuracy:  0.014
Mean F1:  0.729
StDev F1:  0.01


In [155]:
output_model_file = '/content/drive/MyDrive/w266/pytorch_bert_rtgender_easy_data_aug.bin'
output_vocab_file = './'

model_to_save = model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('All files saved')

All files saved


# Parking Lot

labels for sentiment
{'Positive': 2, 'Mixed': 1, 'Neutral': 1, 'Negative':0}

In [ ]:
# create dummy variables for target
train_df = pd.get_dummies(train_df,columns=['sentiment'],drop_first=True, prefix = '', prefix_sep='')
dev_df = pd.get_dummies(dev_df,columns=['sentiment'],drop_first=True, prefix = '', prefix_sep='')

In [ ]:
# convert to list
train_df['list'] = train_df[train_df.columns[7:]].values.tolist()
dev_df['list'] = dev_df[dev_df.columns[7:]].values.tolist()

In [ ]:
new_train = train_df[['response_text', 'list']]
new_dev = dev_df[['response_text', 'list']]

https://colab.research.google.com/github/bhadreshpsavani/ExploringSentimentalAnalysis/blob/main/SentimentalAnalysisWithDistilbert.ipynb

In [ ]:
from transformers import BertTokenizer

model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["response_text"], padding=True, truncation=True)

<a id='section03'></a>
## Preprocess


In [ ]:
# Sections of config
from transformers import BertTokenizer, BertModel, BertConfig

# Defining some key variables that will be used later on in the training
MAX_LEN = 150
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 2
LEARNING_RATE = 1e-05

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case = True,
                                          do_basic_tokenize = True)

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.comment_text = dataframe.response_text
        self.targets = self.data.list
        self.max_len = max_len

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
train_data = new_train.reset_index(drop=True)
dev_data = new_dev.reset_index(drop=True)

print("TRAIN Dataset: {}".format(train_data.shape))
print("DEV Dataset: {}".format(dev_data.shape))

training_set = CustomDataset(train_data, tokenizer, MAX_LEN)
dev_set = CustomDataset(dev_data, tokenizer, MAX_LEN)

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

dev_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
dev_loader = DataLoader(dev_set, **dev_params)

<a id='section04'></a>
### Creating the Neural Network for Fine Tuning

#### Neural Network
 - We will be creating a neural network with the `BERTClass`. 
 - This network will have the BERT Language model followed by a `dropout` and finally a `Linear` layer to obtain the final outputs. 
 - Final layer outputs is what will be compared to the `Sentiment category` to determine the accuracy of models prediction. 
 - We will initiate an instance of the network called `model`. This instance will be used for training and then to save the final trained model for future inference. 
 
#### Loss Function and Optimizer
 - `Loss Function` and `Optimizer` and defined in the next cell.
 - `Optimizer` is used to update the weights of the neural network to improve its performance.

In [ ]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 
%%capture
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased')
        # self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 3)
    
    def forward(self, ids, mask, token_type_ids):
      _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
      # output_2 = self.l2(output_1)
      output = self.l3(output_1)
      return output

model = BERTClass()
model.to(device)

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [ ]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)
        
        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

## Validate

In [ ]:
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(dev_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
f1_score_weighted = []
accuracy = []

for i in range(15):

  for epoch in range(EPOCHS):
      outputs, targets = validation(epoch)
      outputs = np.array(outputs) >= 0.5
      accuracy.append(metrics.accuracy_score(targets, outputs))
      f1_score_weighted(metrics.f1_score(targets, outputs, average='weighted'))
#      print(f"Accuracy Score = {accuracy}")
#      print(f"F1 Score (Weighted) = {f1_score_weighted}")
print(accuracy)
print(f1_score_weighted)    